In [2]:
import numpy as np
import torch
import torchvision

# numpy TEST

In [12]:
# v1 = np.array([1, 0, 0, 1, 2, 3, 5, 0, 0])
# v1 = np.where(v1 != 0)
# print(v1)

# v2 = np.array([[0, 1, 2, 3],
#                [4, 5, 6, 7]])
# v2 = v2.reshape((-1,2,2))
# print(v2)

# v3 = np.array([[[0, 1]],
#                [[2, 7]],
#                [[-1, 0]],
#                [[8, -8]]])
# print(v3.shape)
# print(v3.max(axis=0))
v4 = np.array([0, 1, 2, 3, 4])
print(np.expand_dims(v4, axis=0))
print(v4[:,np.newaxis])

v4 = np.array([[1, 0, 3]])
v5 = np.array([[1, 2, 3],
               [4, 5, 6],
               [7, 8, 9]])
v5 = np.square(v5 - v4)
print(v5)
print(np.sum(v5, axis=-1))
print(np.argsort(np.sum(v5, axis=-1)))

[[0 1 2 3 4]]
[[0]
 [1]
 [2]
 [3]
 [4]]
[[ 0  4  0]
 [ 9 25  9]
 [36 64 36]]
[  4  43 136]
[0 1 2]


# about torch

### 1. DataLoader

In [13]:
import torch.utils.data as data
from torch.utils.data.sampler import Sampler

In [25]:
class MyDataset(data.Dataset):
    def __init__(self):
        self.data = torch.eye(7)
        self.data_len = self.data.size(0)
        
    def __getitem__(self, index):
        print(index)
        return self.data[index]
    
    def __len__(self):
        return int(self.data_len)
    
class Samp(Sampler):
    def __init__(self):
        self.smp = np.array([0, 1, 4, 6, 3, 2, 5])
        self.smp_len = len(self.smp)
    
    def __iter__(self):
        return iter(self.smp)
    
    def __len__(self):
        return self.smp_len


mydataset = MyDataset()    
samp = Samp()

dataloader = data.DataLoader(mydataset,
                             batch_size=3,
                             shuffle=False,
                             sampler=samp)

for i, v in enumerate(dataloader):
    print(v)

0
1
4
tensor([[1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0.]])
6
3
2
tensor([[0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.]])
5
tensor([[0., 0., 0., 0., 0., 1., 0.]])


### 2. Tensor

In [11]:
t1 = torch.randn(3, 3)
print(t1.size(0), type(t1.size(0)))

3 <class 'int'>


### 3. some func api

In [28]:
s1 = torch.randn(3, 3)
s2 = torch.max(s1, 1)
print(s1)
print(s2)

tensor([[-1.0483, -1.0936,  2.4456],
        [ 0.0209, -1.6739, -0.4380],
        [-0.0430, -1.3206, -0.4029]])
torch.return_types.max(
values=tensor([ 2.4456,  0.0209, -0.0430]),
indices=tensor([2, 0, 0]))


# Only Test Something 

In [18]:
import os
print(os.path.join('.', 'asda/asd.jpg'))

./asda/asd.jpg
